# Auxiliary functions

In [2]:
import pandas as pd
pd.read_csv('../metadata_complete/ALL_METADATA.csv')

,Model,Dataset,Reference audio path,Audio path,Speaker ID,Duration reference audio,Duration audio,RTF GPU,RTF CPU,SECS,Input text,Transcription,WER,Evaluator 1 OS,Evaluator 1 Sim-OS,Evaluator 1 real or fake,Evaluator 2 OS,Evaluator 2 Sim-OS,Evaluator 2 real or fake
0,YourTTS,Spanish,audio/Spanish/locutor2_audio_364.wav,synthesized_audios/YourTTS/Spanish/locutor2_au...,locutor2,1.578957,5.429333,0.589072,5.266547,0.811396,La perseverancia es la clave que desbloquea pu...,La perseverancia es la clave que desbloquea p...,0.000000,1.0,NaN,NaN,NaN,NaN,NaN
1,YourTTS,Spanish,audio/Spanish/locutor2_audio_15215_15216.wav,synthesized_audios/YourTTS/Spanish/locutor2_au...,locutor2,6.269388,3.754667,0.088459,0.791379,0.862939,"Vive la vida con intensidad, no con prisa.",Y de la vida con intensidad no comprisa.,0.500000,1.0,NaN,NaN,NaN,NaN,NaN
2,YourTTS,Spanish,audio/Spanish/locutor2_audio_29797.wav,synthesized_audios/YourTTS/Spanish/locutor2_au...,locutor2,5.421859,6.645333,0.088006,0.717888,0.843910,La brisa fresca del parque llevaba consigo ris...,La brisa fresca del parque llevaba con sigo r...,0.166667,NaN,2.0,NaN,NaN,NaN,NaN
3,YourTTS,Spanish,audio/Spanish/locutor2_audio_7682.wav,synthesized_audios/YourTTS/Spanish/locutor2_au...,locutor2,2.925714,9.578667,0.022994,0.195951,0.902871,La música tiene el poder de transportarnos a d...,La música tiene el poder de transportarnos a ...,0.000000,2.0,NaN,NaN,NaN,NaN,NaN
4,YourTTS,Spanish,audio/Spanish/locutor2_audio_18383.wav,synthesized_audios/YourTTS/Spanish/locutor2_au...,locutor2,2.240726,8.853333,0.024146,0.225055,0.793174,Las cicatrices son recordatorios de las batall...,Las cicatrices son recordatorios de las batal...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,TorToise,test,audio/test/1320_122612_000007_000001.wav,synthesized_audios/TorToise/test/1320_122612_0...,1320,4.760000,3.286000,11.847371,111.281342,0.804698,Solihull College is next-door to the east.,Sollyhoe College is next door to the east.,0.428571,NaN,NaN,NaN,NaN,NaN,NaN
1367,TorToise,test,audio/test/1320_122612_000030_000002.wav,synthesized_audios/TorToise/test/1320_122612_0...,1320,9.150000,5.942000,10.663784,97.242127,0.774758,"She is also the mother of Mixcoatl, whom she a...","She is also the mother of Mixcotal, whom she ...",0.076923,4.0,4.0,NaN,NaN,NaN,NaN
1368,TorToise,test,audio/test/1320_122612_000035_000006.wav,synthesized_audios/TorToise/test/1320_122612_0...,1320,4.870000,6.678000,12.511632,108.047729,0.758548,"The unincorporated communities of Goll, McAlli...","The unincorporated communities of goal, McAll...",0.071429,NaN,NaN,NaN,NaN,NaN,NaN
1369,TorToise,test,audio/test/1320_122612_000040_000001.wav,synthesized_audios/TorToise/test/1320_122612_0...,1320,3.800083,5.152667,11.560384,104.348067,0.801076,The horse and buggy was the primary transporta...,The horse and buggy was the primary transport...,0.230769,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import re
from IPython.display import clear_output
import numpy as np

def ask_evaluator():

  metadata_all = pd.read_csv('../metadata_complete/ALL_METADATA.csv')

  evaluators = {re.search(r'Evaluator (\d+)', col).group(0) for col in metadata_all.columns if re.search(r'Evaluator (\d+)', col)}
  
  while True:
    evaluator_id = input("What is your Evaluator ID? (Evaluator 1, Evaluator 2, Evaluator 3...)")

    if evaluator_id in evaluators:
      print('Done')
      break

    elif not evaluator_id:
      evaluator_id = 'Evaluator ' + str(len(evaluators) + 1)
      os_colname = evaluator_id + ' OS'
      sim_os_colname = evaluator_id + ' Sim-OS'
      fake_speech_colname = evaluator_id + ' real or fake'

      metadata_all[os_colname] = np.nan
      metadata_all[sim_os_colname] = np.nan
      metadata_all[fake_speech_colname] = np.nan
      print(f"You have been assigned {evaluator_id}")
      break

    else:
      clear_output(wait=True)
      print('Introduce a valid Evaluator ID')

  metadata_all.to_csv('../metadata_complete/ALL_METADATA.csv', index=False)

  return evaluator_id

In [27]:
import random
from IPython.display import display, Audio

def ask_OS(evaluator_id):

  metadata_all = pd.read_csv('../metadata_complete/ALL_METADATA.csv')
  os_colname = evaluator_id + ' OS'
  # sim_os_colname = evaluator_id + ' Sim-OS'
  valid_opinion = True

  while True:
    # not_evaluated_audios = metadata_all.loc[metadata_all[os_colname].isna(), 'Audio path']
    
    if valid_opinion:

      not_evaluated_audios_indices = metadata_all[metadata_all[os_colname].isnull()].index

      random_index = random.choice(not_evaluated_audios_indices)

      audio_path =  '../' + metadata_all.loc[random_index, 'Audio path']
      # audio_duration = metadata_all.loc[random_index, 'Duration audio']

    clear_output(wait=True)
    display(Audio(audio_path, autoplay=True))
    # print(audio_path)
    # print(metadata_all.loc[random_index, 'Input text'])
    
    if not valid_opinion:
      print('Introduce an integer between 1 and 5 \n')

    print('5: excellent')
    print('4: good')
    print('3: fair')
    print('2: poor')
    print('1: bad')
    print('press q to exit')
    
    # if audio_path.find('synthesized') != -1:
    #   print('Fake')
    # else:
    #   print('Real')
      
    score = input('Rate the voice: ')

    if score == 'q':
      break

    try:

      score = int(score)

      if (score >= 1) and (score <= 5):
        metadata_all.loc[metadata_all['Audio path'] == audio_path, os_colname] = score
        metadata_all.to_csv('../metadata_complete/ALL_METADATA.csv', index=False)
        valid_opinion = True

      else:
        valid_opinion = False

    except:
      valid_opinion = False

In [9]:
def ask_sim_OS(evaluator_id):

  metadata_all = pd.read_csv('../metadata_complete/ALL_METADATA.csv')
  # os_colname = evaluator_id + ' OS'
  sim_os_colname = evaluator_id + ' Sim-OS'
  valid_opinion = True

  while True:
    # not_evaluated_audios = metadata_all.loc[metadata_all[os_colname].isna(), 'Audio path']
    
    if valid_opinion:

      not_evaluated_audios_indices = metadata_all[metadata_all[sim_os_colname].isnull()].index

      random_index = random.choice(not_evaluated_audios_indices)

      audio_path = '../' + metadata_all.loc[random_index, 'Audio path']
      # audio_duration = metadata_all.loc[random_index, 'Duration audio']
      
      a = random.choice([0, 1])
      
      if audio_path.split('_')[0] == '../synthesized':
        reference_audio_path = '../' + metadata_all.loc[random_index, 'Reference audio path']
      
      else:
        speaker_id = metadata_all.loc[random_index, 'Speaker ID']
        metadata_speaker = metadata_all[metadata_all['Speaker ID'] == speaker_id].reset_index(drop=True)
        metadata_speaker = metadata_speaker[metadata_speaker['Model'] == 'Ground truth'].reset_index(drop=True)
        
        reference_audio_path = '../' + metadata_speaker['Audio path'].sample().values[0]

    clear_output(wait=True)
    
    if a == 0:
      display(Audio(audio_path, autoplay=True))
      display(Audio(reference_audio_path))
      
    else:
      display(Audio(reference_audio_path, autoplay=True))
      display(Audio(audio_path))
    
    # print(audio_path.split('_')[0])
    # print(reference_audio_path)
    if not valid_opinion:
      print('Introduce an integer between 1 and 5 \n')

    print('5: excellent')
    print('4: good')
    print('3: fair')
    print('2: poor')
    print('1: bad')
    print('press q to exit')
    score = input('Rate the similarity between both voices: ')

    if score == 'q':
      break

    try:

      score = int(score)

      if (score >= 1) and (score <= 5):
        metadata_all.loc[metadata_all['Audio path'] == audio_path, sim_os_colname] = score
        metadata_all.to_csv('../metadata_complete/ALL_METADATA.csv', index=False)
        valid_opinion = True

      else:
        valid_opinion = False

    except:
      valid_opinion = False

In [6]:
def ask_real_or_fake(evaluator_id):

  metadata_all = pd.read_csv('../metadata_complete/ALL_METADATA.csv')
  real_or_fake_colname = evaluator_id + ' real or fake'
  valid_opinion = True

  while True:
    # not_evaluated_audios = metadata_all.loc[metadata_all[os_colname].isna(), 'Audio path']
    
    if valid_opinion:

      not_evaluated_audios_indices = metadata_all[metadata_all[real_or_fake_colname].isnull()].index

      random_index = random.choice(not_evaluated_audios_indices)

      audio_path = metadata_all.loc[random_index, 'Audio path']
      # audio_duration = metadata_all.loc[random_index, 'Duration audio']

    # print(audio_path)

    clear_output(wait=True)
    display(Audio(audio_path, autoplay=True))
    
    
    if not valid_opinion:
      print('Introduce an integer between 1 and 5 \n')

    print('1: fake')
    print('0: real')
    print('press q to exit')
    score = input('1 = fake, 0 = real: ')

    if score == 'q':
      break

    try:

      score = int(score)

      if (score == 0) or (score == 1):
        metadata_all.loc[metadata_all['Audio path'] == audio_path, real_or_fake_colname] = score
        metadata_all.to_csv('../metadata_complete/ALL_METADATA.csv', index=False)
        valid_opinion = True

      else:
        valid_opinion = False

    except:
      valid_opinion = False

# Evaluator

In [7]:
evaluator_id = ask_evaluator()
evaluator_id

Done


'Evaluator 1'

# Opinion score

In [28]:
ask_OS(evaluator_id)

¡Hola, mi gente! ¿Qué se traen entre manos?
5: excellent
4: good
3: fair
2: poor
1: bad
press q to exit
Fake


# Similarity opinion score

In [84]:
ask_sim_OS(evaluator_id)

../synthesized
../audio/train/1088_134318_000045_000000.wav
5: excellent
4: good
3: fair
2: poor
1: bad
press q to exit


# Real or fake

In [18]:
ask_real_or_fake(evaluator_id)

1: fake
0: real
press q to exit


In [8]:
pd.read_csv('../metadata_complete/ALL_METADATA.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371 entries, 0 to 1370
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Model                     1371 non-null   object 
 1   Dataset                   1371 non-null   object 
 2   Reference audio path      994 non-null    object 
 3   Audio path                1371 non-null   object 
 4   Speaker ID                1371 non-null   object 
 5   Duration reference audio  994 non-null    float64
 6   Duration audio            1371 non-null   float64
 7   RTF GPU                   994 non-null    float64
 8   RTF CPU                   994 non-null    float64
 9   SECS                      994 non-null    float64
 10  Input text                1370 non-null   object 
 11  Transcription             1371 non-null   object 
 12  WER                       1370 non-null   float64
 13  Evaluator 1 OS            571 non-null    float64
 14  Evaluato

Python 3.9.18
